In [1]:
IRT_DIMS = 0
DATASET = 'matrix'    
ITEM_FEATURES = 'True'
ITEM_FEATURES = False if ITEM_FEATURES == 'False' else True

# add layer for reverse
# IRT or use common student embeddings
# common embedding




In [2]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous, create_data_object_heterogeneous_temporal)
import seaborn as sns

from IRT import MIRT_2PL
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

In [3]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 2**11,
#    'neighbours': [-1, -1]
    'neighbours': [30, 30]
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0
    parameters['lambda2'] = 0
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [16,8]
    OUTNAME = 'SAGE' 

if ITEM_FEATURES:
    OUTNAME = OUTNAME + '_scales'
    
print(parameters)
print(DATASET)

{'df_name': None, 'epochs': 10000, 'learning_rate': 0.005, 'weight_decay': 0, 'early_stopping': 200, 'n_splits': 10, 'device': 'cuda:0', 'batch_size': 2048, 'neighbours': [30, 30], 'model_type': 'GNN', 'hidden_dims': [16, 8]}
matrix


In [5]:
DATA_FILE = 'data/mindsteps_set_' + DATASET
df = load_data_heterogeneous(DATA_FILE)

In [8]:
if IRT_DIMS > 0:
        data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True, item_features=ITEM_FEATURES)
    else:
        data, df_student, df_student_age, df_item, df_edge = create_data_object_heterogeneous_temporal(df, return_aux_data=True, item_features=ITEM_FEATURES)

In [15]:
print(data)

HeteroData(
  student={
    node_id=[636327],
    x=[636327, 3]
  },
  item={
    node_id=[17868],
    x=[17868, 11]
  },
  (student, responds, item)={
    edge_index=[2, 5613692],
    edge_attr=[5613692, 2],
    y=[5613692]
  },
  (item, rev_responds, student)={ edge_index=[2, 5613692] },
  (student, preceeds, student)={
    edge_index=[2, 584819],
    edge_attr=[584819, 2]
  }
)


tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 5.1506e+04],
        [0.0000e+00, 1.0000e+00, 5.1507e+04],
        [0.0000e+00, 1.0000e+00, 5.1507e+04]])

%load_ext autoreload
%autoreload
from Heterogeneous_temporal_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
model = EmbedderHeterogeneous( 
    n_students = data['student'].node_id.size(0),
    n_items = data['item'].node_id.size(0),
    student_inchannel = data['student'].x.size(1),
    item_inchannel = data['item'].x.size(1),
    hidden_channels = parameters['hidden_dims'],
    edge_channel = 2,
    metadata = data.metadata()
    ).to(device)
#model(data)

optimizer = torch.optim.Adam(model.parameters(), lr=parameters['learning_rate'], weight_decay=parameters['weight_decay'])
criterion = torch.nn.BCEWithLogitsLoss().to(device)#class_weights)
loss = []
for i in range(3000):
    l = train_embedder_heterogeneous(model, data, optimizer, criterion)
    loss.append(l)
    if i % 100 == 0:
        print(i, l)
        metrics = test_embedder_heterogeneous(model, data, 0, 'test')
        print(metrics)


train_losses = torch.Tensor(loss).detach().numpy()
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
%load_ext autoreload
%autoreload
from manage_experiments import perform_cross_validation
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0it [00:00, ?it/s]

Parameters
{'n_students': 636327, 'n_items': 17868, 'student_inchannel': 3, 'item_inchannel': 11, 'hidden_channels': [16, 8], 'edge_channel': 2}



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/311 [00:00<?, ?it/s]

  2%|▏         | 7/311 [00:10<07:37,  1.51s/it]

  7%|▋         | 21/311 [00:20<04:32,  1.06it/s]

  7%|▋         | 21/311 [00:31<04:32,  1.06it/s]

 12%|█▏        | 38/311 [00:31<03:26,  1.32it/s]

 18%|█▊        | 56/311 [00:41<02:52,  1.48it/s]

 24%|██▍       | 74/311 [00:53<02:35,  1.52it/s]

 29%|██▉       | 91/311 [01:04<02:23,  1.53it/s]

 34%|███▍      | 107/311 [01:14<02:11,  1.55it/s]

 41%|████      | 126/311 [01:24<01:52,  1.65it/s]

 47%|████▋     | 145/311 [01:34<01:37,  1.70it/s]

 53%|█████▎    | 164/311 [01:45<01:25,  1.71it/s]

 59%|█████▉    | 183/311 [01:56<01:12,  1.75it/s]

 65%|██████▍   | 202/311 [02:08<01:04,  1.70it/s]

 70%|███████   | 218/311 [02:18<00:55,  1.67it/s]

 76%|███████▌  | 237/311 [02:28<00:43,  1.72it/s]

 82%|████████▏ | 256/311 [02:40<00:33,  1.67it/s]

 89%|████████▉ | 278/311 [02:50<00:18,  1.81it/s]

100%|██████████| 311/311 [03:10<00:00,  1.63it/s]


{'AUC_0_val': 0.7860767067539696, 'Confusion_0_val': [[77823, 26291], [57635, 118936]], 'Balanced Accuracy_0_val': 0.7105330603410727, 'fold': 0} {'AUC_0_test': 0.7877243718828381, 'Confusion_0_test': [[78054, 26098], [57793, 118740]], 'Balanced Accuracy_0_test': 0.7110230174330475, 'fold': 0}

Epoch: 001, Loss: 0.3940, Val: 0.7105, Test: 0.7110


...


  0%|          | 2/10000 [06:23<530:05:58, 190.87s/it]

  0%|          | 0/311 [00:00<?, ?it/s]

  5%|▌         | 17/311 [00:10<02:58,  1.64it/s]

 11%|█         | 34/311 [00:21<02:55,  1.58it/s]

 16%|█▌        | 50/311 [00:31<02:47,  1.56it/s]

 22%|██▏       | 68/311 [00:42<02:29,  1.62it/s]

 28%|██▊       | 86/311 [00:53<02:21,  1.59it/s]

 33%|███▎      | 102/311 [01:06<02:23,  1.46it/s]

 38%|███▊      | 119/311 [01:17<02:07,  1.51it/s]

 44%|████▎     | 136/311 [01:27<01:52,  1.55it/s]

 44%|████▎     | 136/311 [01:38<01:52,  1.55it/s]

 49%|████▉     | 152/311 [01:38<01:42,  1.55it/s]

 55%|█████▍    | 170/311 [01:48<01:27,  1.61it/s]

 61%|██████    | 189/311 [01:58<01:12,  1.68it/s]

 61%|██████    | 189/311 [02:18<01:12,  1.68it/s]

 66%|██████▌   | 206/311 [02:18<01:20,  1.31it/s]

 73%|███████▎  | 227/311 [02:28<00:56,  1.49it/s]

 80%|███████▉  | 248/311 [02:42<00:41,  1.50it/s]

 87%|████████▋ | 270/311 [02:52<00:24,  1.67it/s]

100%|██████████| 311/311 [03:13<00:00,  

{'AUC_0_val': 0.7906953657701135, 'Confusion_0_val': [[79709, 24405], [59764, 116807]], 'Balanced Accuracy_0_val': 0.7135617034363997, 'fold': 0} {'AUC_0_test': 0.7921856964057153, 'Confusion_0_test': [[80007, 24145], [59802, 116731]], 'Balanced Accuracy_0_test': 0.7147085832856572, 'fold': 0}

Epoch: 003, Loss: 0.3908, Val: 0.7136, Test: 0.7147



  0%|          | 3/10000 [09:37<534:50:11, 192.60s/it]

  0%|          | 0/311 [00:00<?, ?it/s]

  5%|▌         | 16/311 [00:10<03:17,  1.50it/s]

 10%|▉         | 31/311 [00:20<03:08,  1.48it/s]

 15%|█▌        | 47/311 [00:30<02:52,  1.53it/s]

 20%|██        | 63/311 [00:42<02:51,  1.44it/s]

 20%|██        | 63/311 [00:53<02:51,  1.44it/s]

 25%|██▍       | 77/311 [00:53<02:48,  1.39it/s]

 29%|██▉       | 90/311 [01:07<03:04,  1.20it/s]

 32%|███▏      | 100/311 [01:19<03:12,  1.10it/s]

 37%|███▋      | 116/311 [01:29<02:39,  1.22it/s]

 42%|████▏     | 132/311 [01:40<02:16,  1.31it/s]

 48%|████▊     | 148/311 [01:52<02:03,  1.32it/s]

 53%|█████▎    | 164/311 [02:02<01:46,  1.38it/s]

 59%|█████▉    | 184/311 [02:13<01:22,  1.54it/s]

 59%|█████▉    | 184/311 [02:23<01:22,  1.54it/s]

 65%|██████▌   | 203/311 [02:23<01:06,  1.62it/s]

 72%|███████▏  | 223/311 [02:33<00:51,  1.72it/s]

 78%|███████▊  | 243/311 [02:44<00:38,  1.78it/s]

 85%|████████▍ | 264/311 [02:54<00:25,  1.

{'AUC_0_val': 0.7853764285245489, 'Confusion_0_val': [[78918, 25196], [59867, 116704]], 'Balanced Accuracy_0_val': 0.7094713154286505, 'fold': 0} {'AUC_0_test': 0.7864908896894582, 'Confusion_0_test': [[79111, 25041], [59976, 116557]], 'Balanced Accuracy_0_test': 0.7099143519437673, 'fold': 0}




  0%|          | 0/311 [00:00<?, ?it/s]

  5%|▍         | 15/311 [00:10<03:21,  1.47it/s]

 10%|▉         | 30/311 [00:20<03:15,  1.44it/s]

 14%|█▍        | 45/311 [00:31<03:05,  1.44it/s]

 14%|█▍        | 45/311 [00:43<03:05,  1.44it/s]

 19%|█▊        | 58/311 [00:46<03:36,  1.17it/s]

 23%|██▎       | 72/311 [00:56<03:15,  1.23it/s]

 29%|██▊       | 89/311 [01:06<02:43,  1.36it/s]

 34%|███▍      | 106/311 [01:18<02:26,  1.40it/s]

 40%|███▉      | 124/311 [01:28<02:03,  1.51it/s]

 46%|████▌     | 142/311 [01:39<01:49,  1.55it/s]

 51%|█████     | 159/311 [01:50<01:37,  1.55it/s]

 57%|█████▋    | 177/311 [02:00<01:22,  1.62it/s]

 63%|██████▎   | 195/311 [02:11<01:10,  1.64it/s]

 63%|██████▎   | 195/311 [02:23<01:10,  1.64it/s]

 68%|██████▊   | 210/311 [02:24<01:08,  1.48it/s]

 75%|███████▍  | 233/311 [02:34<00:46,  1.69it/s]

 82%|████████▏ | 256/311 [02:45<00:30,  1.79it/s]

 89%|████████▉ | 277/311 [02:55<00:18,  1.87it/s]

In [26]:
output_dict.keys()
print('AUC:', output_dict['AUC_0_test'])
print('Balanced Accuracy:', output_dict['Balanced Accuracy_0_test'])

NameError: name 'output_dict' is not defined

In [ ]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data.cpu(), train_data.cpu(), val_data.cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_{DATASET}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}_{DATASET}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix
if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['offset']
    df_item['IRT1_discrimination'] = z_dict['discrimination']
    df_item['IRT1_discrimination_transf'] = z_dict['item']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data.cpu(), train_data.cpu(), val_data.cpu().cpu(), test_data.cpu(),
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_{DATASET}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])    
    
    fig = plt.figure()
    sns.scatterplot(x='IRT_difficulty', y='IRT1_difficulty', data=df_item, hue='scale')
    plt.title('Difficulty')
    print('Difficulty:', pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    sns.scatterplot(x='age', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Age-Ability')
    
    fig = plt.figure()
    sns.scatterplot(x='grade', y='IRT1_ability', data=df_edge_clean, hue='age')
    plt.title('Grade-Ability')
    print('Age-Ability:', pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    sns.scatterplot(x='ability', y='IRT1_ability', data=df_edge_clean, hue='grade')
    plt.title('Ability')
    print('Ability:', pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    